# Filings Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

monthly total before running time.

In [ ]:
%%sql stat_filings_monthly_completed  <<
WITH Detail AS 
(
    SELECT le.identifier AS COOPERATIVE_NUMBER
           , le.legal_name AS COOPERATIVE_NAME
           , COUNT(le.identifier) AS FILINGS_TOTAL_COMPLETED
           , STRING_AGG(f.filing_type, ', ')  AS FILING_TYPES_COMPLETED
    FROM legal_entities le,
    filings f
    WHERE le.id = f.legal_entity_id
    AND le.entity_type='CP'
    AND f.status='COMPLETED'
    AND date(f.completion_date at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 months')    
    AND date(f.completion_date at time zone 'utc' at time zone 'pst')  <= date(current_date - 1)
    GROUP BY le.identifier, le.legal_name    
)

SELECT * FROM Detail
UNION ALL
SELECT 'SUM' identifier, null, sum(FILINGS_TOTAL_COMPLETED) AS count, null
from Detail;

In [ ]:
edt = stat_filings_monthly_completed.DataFrame()

Save to CSV

In [ ]:
last_month = datetime.now() - relativedelta(months=1)
filename = os.path.join(os.getcwd(), r'data/')+'coop_filings_monthly_stats_for_' + format(last_month, '%B_%Y') +'.csv'
edt.to_csv(filename, sep=',', encoding='utf-8', index=False)